In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00


In [ ]:
!pip install pytorch_lightning

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
from torchvision.transforms import Resize
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Normalize, Compose

height = 600
width = 800

# Define dataset transforms including resizing
transform = Compose([Resize((224, 224)), ToTensor(), Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

# Load the iNaturalist dataset
dataset = ImageFolder(root='/content/inaturalist_12K', transform=transform)

# Split the dataset into train and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Define dataloaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import wandb

class CNNModel(pl.LightningModule):
    def __init__(self, input_shape, num_classes, num_filters=32, filter_size=(3, 3), activation='relu', dense_neurons=128):
        super().__init__()
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.activation = activation
        self.dense_neurons = dense_neurons

        self.conv_layers = nn.Sequential(
            nn.Conv2d(input_shape[0], num_filters, filter_size, padding='same'),
            self.get_activation(),
            nn.MaxPool2d(2),
            nn.Conv2d(num_filters, num_filters , filter_size, padding='same'),
            self.get_activation(),
            nn.MaxPool2d(2),
            nn.Conv2d(num_filters , num_filters , filter_size, padding='same'),
            self.get_activation(),
            nn.MaxPool2d(2),
            nn.Conv2d(num_filters , num_filters , filter_size, padding='same'),
            self.get_activation(),
            nn.MaxPool2d(2),
            nn.Conv2d(num_filters , num_filters, filter_size, padding='same'),
            self.get_activation(),
            nn.MaxPool2d(2)
        )

        self.dense_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.conv_output_size(), dense_neurons),
            self.get_activation(),
            nn.Linear(dense_neurons, num_classes)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.dense_layers(x)
        return x

    def get_activation(self):
        if self.activation == 'relu':
            return nn.ReLU()
        elif self.activation == 'leaky_relu':
            return nn.LeakyReLU()

    def conv_output_size(self):
        dummy_input = torch.zeros(1, *self.input_shape)
        dummy_output = self.conv_layers(dummy_input)
        return dummy_output.flatten(1).size(-1)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

# Define parameters for convolution layers and dense layers
input_shape = (3, 224, 224)
num_classes = 10
model = CNNModel(input_shape, num_classes)

# Initialize and log the model
wandb.init(project='DL_Assign_1')  # Replace 'wandb_username' with your Wandb username
wandb_logger = pl.loggers.WandbLogger()
wandb_logger.watch(model)
wandb_logger.log_hyperparams({'conv_params': conv_params, 'dense_params': dense_params})

# Define dataloaders (Replace train_dataset and val_dataset with your actual datasets)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Define PyTorch Lightning trainer
trainer = pl.Trainer(logger=wandb_logger, max_epochs=10)
trainer.fit(model, train_loader, val_loader)


/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type       | Params
--------------------------------------------
0 | conv_layers  | Sequential | 37.9 K
1 | dense_layers | Sequential | 202 K 
--------------------------------------------
240 K     Trainable params
0         Non-trainable params
240 K     Total params
0.960   

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=10` reached.
